In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
from environmental_risk_metrics import Sentinel2
from IPython.display import Image
import geopandas as gpd

# Configure logging to display in Jupyter
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    force=True,  # This ensures the configuration is applied even if logging was previously configured
)
logger = logging.getLogger(__name__)



polygon = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
            [
                [10.235198982658801, 51.42076009745068],
                [10.236477278753114, 51.41697045550828],
                [10.244461712820623, 51.41823370440062],
                [10.242888425319222, 51.4220355049745],
                [10.235198982658801, 51.42076009745068],
            ]
        ],
        "type": "Polygon",
    },
}
start_date="2024-06-27"
end_date="2024-12-31"

gdf = gpd.GeoDataFrame.from_features([polygon], crs="EPSG:4326")

sentinel2 = Sentinel2(
    gdf=gdf,
    start_date=start_date,
    end_date=end_date,
)

sentinel2.get_items()
sentinel2.load_xarray()
sentinel2.load_ndvi_images()
sentinel2.ndvi_thumbnails()
data = sentinel2.calculate_mean_ndvi()

Fetching sentinel-2-l2a items:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

In [7]:
data["sentinel-2-l2a"][0].dropna()

ndvi  interpolated_ndvi  is_bare_soil
2024-07-09  0.290114           0.290114         False
2024-07-29  0.140536           0.140536          True
2024-08-06  0.131831           0.131831          True
2024-08-13  0.140835           0.140835          True
2024-08-16  0.130911           0.130911          True
2024-08-28  0.111633           0.111633          True
2024-09-02  0.119948           0.119948          True
2024-09-05  0.124700           0.124700          True
2024-09-20  0.429606           0.429606         False
2024-10-25  0.650251           0.650251         False
2024-12-01  0.638912           0.638912         False

In [ ]:
import logging
from environmental_risk_metrics.metrics.ndvi import HarmonizedNDVI
from IPython.display import Image
import geopandas as gpd

# Configure logging to display in Jupyter
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    force=True,  # This ensures the configuration is applied even if logging was previously configured
)
logger = logging.getLogger(__name__)

# Same polygon as before
polygon = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
            [
                [10.235198982658801, 51.42076009745068],
                [10.236477278753114, 51.41697045550828],
                [10.244461712820623, 51.41823370440062],
                [10.242888425319222, 51.4220355049745],
                [10.235198982658801, 51.42076009745068],
            ]
        ],
        "type": "Polygon",
    },
}
start_date="2024-06-27"
end_date="2024-12-31"

gdf = gpd.GeoDataFrame.from_features([polygon], crs="EPSG:4326")

# Use HarmonizedNDVI with multiple collections
harmonized_ndvi = HarmonizedNDVI(
    gdf=gdf,
    start_date=start_date,
    end_date=end_date,
    collections=["hls2-s30"],  # Multiple collections
    resolution=30,  # Using 30m resolution for consistency across collections
)

# Get items for all collections
harmonized_ndvi.get_items()
# Load xarray data
harmonized_ndvi.load_xarray()
# Load NDVI images
harmonized_ndvi.load_ndvi_images()
# Generate thumbnails
harmonized_ndvi.ndvi_thumbnails()
# Calculate mean NDVI
harmonized_data = harmonized_ndvi.calculate_mean_ndvi()


Fetching hls2-s30 items:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

In [ ]:
harmonized_ndvi.gdf.geometry[0].__geo_interface__

In [31]:
import odc

ds = odc.stac.load(
    harmonized_ndvi.items["hls2-s30"][0],
    resolution=30,
    geopolygon=harmonized_ndvi.gdf.geometry[0].__geo_interface__,
)

ds

<xarray.Dataset> Size: 729kB
Dimensions:      (y: 20, x: 23, time: 22)
Coordinates:
  * y            (y) float64 160B 5.697e+06 5.697e+06 ... 5.697e+06 5.697e+06
  * x            (x) float64 184B 5.859e+05 5.859e+05 ... 5.865e+05 5.865e+05
    spatial_ref  int32 4B 32632
  * time         (time) datetime64[ns] 176B 2024-07-09T10:26:35.814400 ... 20...
Data variables: (12/18)
    B01          (time, y, x) float32 40kB 278.0 248.0 248.0 ... 230.0 230.0
    B02          (time, y, x) float32 40kB 323.0 319.0 334.0 ... 290.0 321.0
    B03          (time, y, x) float32 40kB 754.0 690.0 762.0 ... 534.0 592.0
    B04          (time, y, x) float32 40kB 474.0 511.0 610.0 ... 879.0 924.0
    B05          (time, y, x) float32 40kB 1.165e+03 1.054e+03 ... 1.1e+03
    B06          (time, y, x) float32 40kB 3.394e+03 2.98e+03 ... 1.268e+03
    ...           ...
    B8A          (time, y, x) float32 40kB 4.422e+03 3.968e+03 ... 1.51e+03
    SAA          (time, y, x) float32 40kB 1.536e+04 1.536e+04 ... 1.7e+04
    SZA          (time, y, x) float32 40kB 3.121e+03 3.121e+03 ... 7.388e+03
    VAA          (time, y, x) float32 40kB 9.512e+03 9.512e+03 ... 9.588e+03
    VZA          (time, y, x) float32 40kB 811.0 811.0 811.0 ... 807.0 807.0
    Fmask        (time, y, x) float32 40kB 64.0 64.0 64.0 ... 128.0 64.0 64.0

In [ ]:




for collection in self.collections:
    self.xarray_data[collection] = []
    items_list = items_dict.get(collection, [])
    
    if not items_list:
        logger.warning(f"No items found for collection {collection}")
        continue
        
    # Get bands for this collection
    if bands and collection in bands:
        collection_bands = bands[collection]
    else:
        collection_bands = self._get_bands_for_collection(collection, include_rgb)
    
    mapping = self.BAND_MAPPINGS[collection]
    
    for geometry_idx, (items, geometry) in enumerate(zip(items_list, self.gdf.geometry)):
        if not items:
            logger.warning(f"No items for geometry {geometry_idx} in collection {collection}")
            self.xarray_data[collection].append(None)
            continue
            
        # Sign the items to get access
        logger.debug(f"Signing {len(items)} items for collection {collection}")
        signed_items = [planetary_computer.sign(i) for i in items]

        thread_pool = ThreadPoolExecutor(max_workers=self.max_workers)

        # Load the data into an xarray Dataset
        logger.debug(f"Loading data into xarray Dataset for {collection}")
        progress = tqdm if show_progress else None
        
        try:
            ds = odc.stac.load(
                items=signed_items,
                bands=collection_bands,
                resolution=self.resolution,
                pool=thread_pool,
                geopolygon=geometry,
                progress=progress,
            )
        except Exception as e:
            logger.error(f"Failed to load data for {collection}, geometry {geometry_idx}: {e}")
            self.xarray_data[collection].append(None)
            continue

        # Apply cloud filtering if requested and cloud mask is available
        if self.cropped_image_cloud_cover_threshold and filter_cloud_cover and mapping["cloud_mask"]:
            logger.debug(f"Filtering data based on cloud cover using {mapping['cloud_mask']} band")
            
            if collection == "sentinel-2-l2a":
                # For Sentinel-2, use SCL band
                cloud_clear_mask = (ds[mapping["cloud_mask"]] == 4) | (ds[mapping["cloud_mask"]] == 5)
            else:
                # For HLS collections, use Fmask band
                cloud_clear_mask = ds[mapping["cloud_mask"]] == 0
            
            cloud_cover_pct = (1 - cloud_clear_mask.mean(dim=["x", "y"])) * 100
            logger.debug(f"Cloud cover percentage: {cloud_cover_pct}. Filtering based on {self.cropped_image_cloud_cover_threshold}% threshold")
            
            logger.debug(f"Dataset time steps before filtering: {len(ds.time)}")
            ds = ds.sel(time=cloud_cover_pct <= self.cropped_image_cloud_cover_threshold)
            logger.debug(f"Filtered dataset to {len(ds.time)} time steps")

            # Apply spatial cloud masking
            if filter_cloud_cover:
                ds = ds.where(cloud_clear_mask, drop=False)

        logger.debug(f"Successfully loaded {collection} data")
        self.xarray_data[collection].append(ds)
        

In [19]:
harmonized_ndvi.xarray_data["hls2-s30"][0]

<xarray.Dataset> Size: 348B
Dimensions:      (time: 0, y: 20, x: 23)
Coordinates:
  * y            (y) float64 160B 5.697e+06 5.697e+06 ... 5.697e+06 5.697e+06
  * x            (x) float64 184B 5.859e+05 5.859e+05 ... 5.865e+05 5.865e+05
  * time         (time) datetime64[ns] 0B 
    spatial_ref  int32 4B 32632
Data variables:
    B08          (time, y, x) float32 0B 
    B04          (time, y, x) float32 0B 
    Fmask        (time, y, x) float32 0B

In [9]:
# Display results from all collections
for collection in harmonized_data.keys():
    print(f"\n=== {collection.upper()} ===")
    df = harmonized_data[collection][0].dropna()
    if not df.empty:
        print(df)
        print(f"Mean NDVI: {df['ndvi'].mean():.3f}")
        print(f"Data points: {len(df)}")
    else:
        print("No data available")



=== SENTINEL-2-L2A ===
                ndvi  interpolated_ndvi  is_bare_soil
2024-07-09  0.308067           0.308067         False
2024-07-29  0.172118           0.172118          True
2024-08-06  0.161856           0.161856          True
2024-08-13  0.170007           0.170007          True
2024-08-16  0.160726           0.160726          True
2024-08-28  0.142951           0.142951          True
2024-09-02  0.148638           0.148638          True
2024-09-05  0.146268           0.146268          True
2024-09-20  0.417695           0.417695         False
2024-10-25  0.626452           0.626452         False
2024-12-01  0.617615           0.617615         False
Mean NDVI: 0.279
Data points: 11

=== HLS2-S30 ===
No data available

=== HLS2-L30 ===
No data available
